### ANN to predict the news sentiment
Description: Based on "open", "close", Moving Average "MA" and "adj close" data

The purpose: to fill in the missing data on news sentiment

In [1]:
#import libraries
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from pandas import datetime
import math
from math import sqrt
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import mplfinance as mpf
import matplotlib.dates as mdates
from news_test import daily_sentiment #NLP component
plt.style.use('seaborn-deep')
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split

Read the data as pandas dataframe

Use the dataset from data_clean that has the data of the stock market starting from 2015-01-01

In [2]:
ticker = 'AAPL'
df = pd.read_csv('./data_clean/' + ticker + '.csv')
#turn index col (string) into date col

df = df.query('date <= "2020,1,1"') 
df = df.reset_index(drop=True)

Create a news sentiment column, in which put the accumulated sentiment over the last 5 days

In [3]:
df['sentiment'] = 0.0


In [4]:
#add a sentiment
sent_df = pd.read_csv('news_sentiments.csv')
sent_df = sent_df.loc[sent_df['ticker'] == ticker]

for i, _ in df.iterrows():
    s = df['date'][i]
    e = datetime.datetime.strptime(s, "%Y-%m-%d").date() - datetime.timedelta(days=5)
    sent = sent_df.query('release_date < "' + s + '" and release_date >= "' + e.strftime("%Y-%m-%d") + '"')
    df['sentiment'][i] = (sum(sent['sentiment']) / len(sent)) if len(sent) else 0.0

In [5]:
df

,date,open,high,low,close,adj close,volume,sentiment
0,2015-01-02,111.389999,111.440002,107.349998,109.330002,100.216454,53204600,0.400038
1,2015-01-05,108.290001,108.650002,105.410004,106.250000,97.393181,64285500,0.448022
2,2015-01-06,106.540001,107.430000,104.629997,106.260002,97.402374,65797100,0.624988
3,2015-01-07,107.199997,108.199997,106.699997,107.750000,98.768150,40105900,0.164748
4,2015-01-08,109.230003,112.150002,108.699997,111.889999,102.563072,59364500,0.181473
...,...,...,...,...,...,...,...,...
1253,2019-12-24,284.690002,284.890015,282.920013,284.269989,283.596924,12119700,0.164095
1254,2019-12-26,284.820007,289.980011,284.700012,289.910004,289.223602,23280300,0.190640
1255,2019-12-27,291.119995,293.970001,288.119995,289.799988,289.113831,36566500,0.258179
1256,2019-12-30,289.459991,292.690002,285.220001,291.519989,290.829773,36028600,1.126792


In [6]:
#function to calculate the technical indicator - moving average
def moving_average(df, n):
   
    MA = pd.Series(df['close'].rolling(n, min_periods=n).mean(), name='MA' )
    df = df.join(MA)
    return df
df_MA = moving_average(df, 5) # moving average based on the last 5 days
#change the order of columns so that adj close is the last column
columns_titles = ["open","high", "low", "close", "volume",  "MA", "adj close", "sentiment",]
df_MA=df_MA.reindex(columns=columns_titles)
df_MA =df_MA.dropna() # keep only the rows with data
df_MA.describe()


,open,high,low,close,volume,MA,adj close,sentiment
count,1254.000000,1254.000000,1254.000000,1254.000000,1.254000e+03,1254.000000,1254.000000,1254.000000
mean,154.544242,155.905566,153.225096,154.626738,3.583632e+07,154.332151,149.256203,0.147896
std,44.002469,44.404753,43.690745,44.109509,1.786791e+07,43.751460,45.916808,0.222272
min,90.000000,91.669998,89.470001,90.339996,1.136200e+07,91.915999,85.192909,-0.742159
25%,115.922499,116.577501,114.730001,115.597498,2.377555e+07,115.776500,108.258678,0.026317
50%,150.715004,152.885002,149.425003,150.739998,3.155720e+07,151.642999,146.311195,0.125172
75%,186.107498,187.602497,184.699997,186.102497,4.255100e+07,186.203001,182.131222,0.232805
max,291.119995,293.970001,289.519989,293.649994,1.622063e+08,289.829993,292.954712,1.933697


In [7]:
#one-hot encoding of the news sentiment
#if the news sentiment is positive, assign to it 1, if negative, assign 0
for idx, row in df_MA.iterrows():
    if  df_MA.loc[idx,'sentiment'] >0:
        df_MA.loc[idx,'sentiment'] = 1
    
    else:
        
        df_MA.loc[idx,'sentiment'] = 0

#drop redundant features and select the features you want to test the prediction on
df_MA.drop(['volume'], inplace = True, axis = 1)
df_MA.drop(['low'], inplace = True, axis = 1)
df_MA.drop(['high'], inplace = True, axis = 1)

Train test split. Our labels are sentiments obtained after hot encoding

In [8]:


X = df_MA.drop(labels='sentiment', axis =1)

y = df_MA['sentiment']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False)


Standardize the data to perform prediction

In [10]:
scaler = StandardScaler()  

# fit only on the training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  

# apply same transformation to test data
X_test = scaler.transform(X_test)

Hypermarameter tuning - the search of the optimal model to perform the classification

In [11]:
#The code on performing hypermarameter tuning was provided to us by Prof. Raja in WB 14
%%time

param_grid = {'hidden_layer_sizes': [(3,), (5,), (10,)], 'solver':['sgd', 'lbfgs'], 
              'alpha': (0.1, 0.01),'activation': ['logistic', 'relu'], 'tol': (0.001, 0.0001),
             'learning_rate': ['adaptive', 'constant']}

clf_mlp = MLPClassifier()

clf_mlp_cv = GridSearchCV(clf_mlp, param_grid, scoring='accuracy', cv=5, verbose=1, n_jobs=-1)
clf_mlp_cv.fit(X_train, y_train)


params_optimal_mlp = clf_mlp_cv.best_params_

print("Best Score (accuracy): %f" % clf_mlp_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal_mlp)
print("\n")

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:    6.2s


Best Score (accuracy): 0.793622
Optimal Hyperparameter Values:  {'activation': 'logistic', 'alpha': 0.1, 'hidden_layer_sizes': (3,), 'learning_rate': 'adaptive', 'solver': 'sgd', 'tol': 0.001}


CPU times: user 786 ms, sys: 74.4 ms, total: 860 ms
Wall time: 11.2 s


[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   11.0s finished


Training of the model using the optimal parameters

In [12]:
%%time
mlp_clf = MLPClassifier(random_state=42, hidden_layer_sizes=(3, ), alpha=0.1, solver='sgd',
                      max_iter=200, learning_rate='adaptive', activation='logistic',
                    verbose=True)


mlp_clf.fit(X_train, y_train)

Iteration 1, loss = 0.85384195
Iteration 2, loss = 0.84524804
Iteration 3, loss = 0.83245698
Iteration 4, loss = 0.81771909
Iteration 5, loss = 0.80321761
Iteration 6, loss = 0.78902338
Iteration 7, loss = 0.77574676
Iteration 8, loss = 0.76421419
Iteration 9, loss = 0.75245887
Iteration 10, loss = 0.74106700
Iteration 11, loss = 0.73075912
Iteration 12, loss = 0.72080366
Iteration 13, loss = 0.71065604
Iteration 14, loss = 0.70129551
Iteration 15, loss = 0.69261816
Iteration 16, loss = 0.68388207
Iteration 17, loss = 0.67546571
Iteration 18, loss = 0.66733036
Iteration 19, loss = 0.66016469
Iteration 20, loss = 0.65315510
Iteration 21, loss = 0.64661552
Iteration 22, loss = 0.63990422
Iteration 23, loss = 0.63344529
Iteration 24, loss = 0.62741242
Iteration 25, loss = 0.62164410
Iteration 26, loss = 0.61600678
Iteration 27, loss = 0.61087389
Iteration 28, loss = 0.60620055
Iteration 29, loss = 0.60241173
Iteration 30, loss = 0.59856053
Iteration 31, loss = 0.59428638
Iteration 32, los

MLPClassifier(activation='logistic', alpha=0.1, hidden_layer_sizes=(3,),
              learning_rate='adaptive', random_state=42, solver='sgd',
              verbose=True)

In [13]:
## MLP: Evaluation on Training Data

In [14]:
print("No. of Iterations:", mlp_clf.n_iter_ )

y_train_predicted = mlp_clf.predict(X_train)

train_accuracy_mlp = np.mean(y_train_predicted == y_train)
print("\nTraining Accuracy: ", train_accuracy_mlp)

No. of Iterations: 175

Training Accuracy:  0.7936191425722832


MLP: Evaluation on Test Data

In [15]:
y_test_predicted = mlp_clf.predict(X_test)

test_accuracy_mlp = np.mean(y_test_predicted == y_test)
print("\nTest Accuracy: ", test_accuracy_mlp)


print("\nTest Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted))


precision_test = precision_score(y_test, y_test_predicted) 
print("\nTest Precision = %f" % precision_test)

recall_test = recall_score(y_test, y_test_predicted)
print("Test Recall = %f" % recall_test)


f1_test = f1_score(y_test, y_test_predicted)
print("Test F1 Score = %f" % f1_test)


print("\nClassification Report:")
print(classification_report(y_test, y_test_predicted))


Test Accuracy:  0.8286852589641435

Test Confusion Matrix:
[[  0  43]
 [  0 208]]

Test Precision = 0.828685
Test Recall = 1.000000
Test F1 Score = 0.906318

Classification Report:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        43
         1.0       0.83      1.00      0.91       208

    accuracy                           0.83       251
   macro avg       0.41      0.50      0.45       251
weighted avg       0.69      0.83      0.75       251



Overall, ANN_predict_sentiment is a way for us to fill in the missing data in the news sentiment so that our SVM, RF, LSTM models could be applied to the years for which we do not have the news sentiment database